# GET NEXT LINE

## get_next_line.h

In [53]:
%%file ../gnl_working/get_next_line.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line.h                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:29 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:32 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef GET_NEXT_LINE_H_
# define GET_NEXT_LINE_H_

char	*get_next_line(int fd);

#endif

Overwriting ../gnl_working/get_next_line.h


## ~M~A~N~D~A~T~O~R~Y~

### get_next_line.c

In [91]:
%%file ../gnl_working/get_next_line.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line.c                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:18 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:23 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "get_next_line.h"

char	*get_next_line(int fd)
{

	return (0);
}


Overwriting ../gnl_working/get_next_line.c


### get_next_line_utils.c

In [88]:
%%file ../gnl_working/get_next_line_utils.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line_utils.c                              :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:38 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:41 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "get_next_line.h"


Overwriting ../gnl_working/get_next_line_utils.c


### gnl_test

In [92]:
%%file gnl_test.c
#include "../gnl_working/get_next_line.h"
#include <stdio.h>
#include <stdlib.h>

int main (void)
{
	get_next_line();

	return (0);
}

Overwriting gnl_test.c


In [94]:
%%bash
cc -Wall -Wextra -Werror -D BUFFER_SIZE=42 gnl_test.c ../gnl_working/get_next_line.c ../gnl_working/get_next_line_utils.c -o gnl_test
./gnl_test

## ~B~O~N~U~S~

- Add bonus material here

## Makefile

### Makefile

In [14]:
%%file ../gnl_testing/Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2022/12/01 14:36:56 by dpentlan          #+#    #+#              #
#    Updated: 2022/12/01 14:36:58 by dpentlan         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

# Not needed for get next line. But in case I want to use this later it's here.
# Note that is not working yet..

NAME = gnl

SRCS =	get_next_line_utils.c \
	get_next_line.c

BONUS_SRCS =

OBJS = ${SRCS:.c=.o}

BONUS_OBJS = ${BONUS_SRCS:.c=.o}

INCS = get_next_line.h

CFLAGS = -Wall -Wextra -Werror

CC = cc

SLIB = ar rcs

all: $(NAME)

$(NAME): $(OBJS) $(INCS)
	@$(SLIB) $(NAME) -o $(OBJS)

bonus: $(OBJS) $(BONUS_OBJS) $(INCS)
	@$(SLIB) $(NAME) -o $(OBJS) $(BONUS_OBJS)

%.o: %.c
	@$(CC) ${CFLAGS} -c $< -o ${<:.c=.o}

clean:
	@rm -f $(OBJS) $(BONUS_OBJS)

fclean:	clean
	@rm -f $(NAME)

re:	fclean all


Overwriting ../gnl_testing/Makefile


### Build Mandatory

In [ ]:
%%bash
cd ../gnl_working
# add build commands here